In [1]:
"""
Import
"""
from matplotlib.colors import ListedColormap, BoundaryNorm, TwoSlopeNorm
import matplotlib.cm as cm
import matplotlib.lines as mlines
from typing import List

import numpy as np
import pandas as pd
import xarray as xr
import pickle
import os
import glob
import sys
from datetime import datetime,timedelta
import matplotlib.patches as patches
from matplotlib.path import Path
import matplotlib.pyplot as plt
import numpy.ma as ma
import cartopy.crs as ccrs  # Import cartopy ccrs
import cartopy.feature as cfeature  # Import cartopy common features
from matplotlib import animation
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error
import scipy.stats
import math
import time as time_mod
sys.path.insert(0, "/home/chalifour/code/master")
from fct_script.func_py import get_proj_extent
import fct_script.rpn_funcs_chris as rpn_chris
from fct_script.get_domain import get_domain_info
from fct_script.func_py import get_colormap_precip,categorical_cmap
from matplotlib.patches import Patch
# from func_py import get_proj_extent

try:
    import rpnpy.librmn.all as rmn  # Module to read RPN files
    from rotated_lat_lon import RotatedLatLon  # Module to project field on native grid (created by Sasha Huziy)
except ImportError as err:
    print(f"RPNPY can only be use on the server. It can't be use on a personal computer."
          f"\nError throw :{err}")

In [2]:
m, lonE2p5, latE2p5 = get_proj_extent()

{}
263.0 47.5 353.0 0.0


---
---
# Time array


In [3]:


begin,end = '2021-10','2022-06'
begin_date,end_date  = datetime.strptime(begin, '%Y-%m'),datetime.strptime(end, '%Y-%m')

timerange_month = pd.date_range(begin,end,freq='MS')
timerange_day = pd.date_range(begin,end,freq='D')


season = ['ON','DJF','MAM']
season_dict= {'ON':{'month':[10,11],'year':begin_date.year},
              'DJF':{'month':[12,1,2],'year':begin_date.year},
              'MAM':{'month':[3,4,5],'year':end_date.year}}




---
---
# Important path

In [4]:

# ON
path_daymet_tt_on= sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{begin_date.year}/*1[0-1]/daymet_1km_v4r1_tavg_lc_{begin_date.year}*1[0-1]_d.nc4'))
path_daymet_pr_on = sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{begin_date.year}/*1[0-1]/daymet_1km_v4r1_prcp_lc_{begin_date.year}*1[0-1]_d.nc4'))

# path_sim_11km_on = sorted(glob.glob(fr'/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_{begin_date.year}*1[0-1]'))
path_sim_11km_on = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{begin_date.year}*1[0-1]'))

# path_sim_2p5km_on = sorted(glob.glob(fr'/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_{begin_date.year}*1[0-1]'))
path_sim_2p5km_on = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{begin_date.year}*1[0-1]'))



# DJF
# path_daymet_tt_djf= sorted(glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_ta*_lc_{begin_date.year}12_d.nc4') + glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{end_date.year}/0*[1-2]/daymet_1km_v4r1_ta*_lc_{end_date.year}0*[1-2]_d.nc4'))
path_daymet_tt_djf= sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_tavg_lc_{begin_date.year}12_d.nc4') + glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{end_date.year}/0*[1-2]/daymet_1km_v4r1_ta*_lc_{end_date.year}0*[1-2]_d.nc4'))

# path_daymet_pr_djf = sorted(glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_prcp_lc_{begin_date.year}12_d.nc4')+glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{end_date.year}/0*[1-2]/daymet_1km_v4r1_prcp_lc_{end_date.year}0*[1-2]_d.nc4'))

path_daymet_pr_djf = sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_prcp_lc_{begin_date.year}12_d.nc4')+glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{end_date.year}/0*[1-2]/daymet_1km_v4r1_prcp_lc_{end_date.year}0*[1-2]_d.nc4'))

# path_daymet_tt_djf= sorted(glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_ta*_lc_{begin_date.year}12_d.nc4') + glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/ta*/lc/nc4/2022/0*[1-2]/daymet_1km_v4r1_ta*_lc_{end_date.year}0*[1-2]_d.nc4'))
# path_daymet_pr_djf = sorted(glob.glob(
#             fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{begin_date.year}/12/daymet_1km_v4r1_prcp_lc_{begin_date.year}12_d.nc4')+glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/prcp/lc/nc4/2022/0*[1-2]/daymet_1km_v4r1_prcp_lc_{end_date.year}0*[1-2]_d.nc4'))


# path_sim_11km_djf = sorted(glob.glob(fr'/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_{begin_date.year}12')+glob.glob(fr'/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_{end_date.year}0*[1-2]'))
path_sim_11km_djf = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{begin_date.year}12')+glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{end_date.year}0*[1-2]'))

# path_sim_2p5km_djf = sorted(glob.glob(fr'/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_{begin_date.year}12')+glob.glob(fr'/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_{end_date.year}0*[1-2]'))
path_sim_2p5km_djf = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{begin_date.year}12')+glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{end_date.year}0*[1-2]'))

# /pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll
# MAM
path_daymet_tt_mam= sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/ta*/lc/nc4/{end_date.year}/0*[3-5]/daymet_1km_v4r1_tavg_lc_{end_date.year}0*[3-5]_d.nc4'))
path_daymet_pr_mam = sorted(glob.glob(
            fr'/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/prcp/lc/nc4/{end_date.year}/0*[3-5]/daymet_1km_v4r1_prcp_lc_{end_date.year}0*[3-5]_d.nc4'))
#
# path_daymet_tt_mam= sorted(glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/ta*/lc/nc4/{end_date.year}/0*[3-5]/daymet_1km_v4ll_ta*_lc_{end_date.year}0*[3-5]_d.nc4'))
#
# path_daymet_pr_mam = sorted(glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/prcp/lc/nc4/{end_date.year}/0*[3-5]/daymet_1km_v4ll_prcp_lc_{end_date.year}0*[3-5]_d.nc4'))

# path_daymet_tt_mam= sorted(glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/ta*/lc/nc4/2022/0*[3-5]/daymet_1km_v4ll_ta*_lc_{end_date.year}0*[3-5]_d.nc4'))
# path_daymet_pr_mam = sorted(glob.glob(
#             fr'/pampa/roberge/OBSERVATIONS/Daymet_1km_v4ll/daily/prcp/lc/nc4/2022/0*[3-5]/daymet_1km_v4ll_prcp_lc_{end_date.year}0*[3-5]_d.nc4'))
#
# path_sim_11km_mam = sorted(glob.glob(fr'/BIG1/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs/Samples/NAM-11m_ERA5_GEM5_CLASS_NV_NA_newP3-SCPF_SN8_20yrs_{end_date.year}0*[3-5]'))
path_sim_11km_mam = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil_{end_date.year}0*[3-5]'))
# /chinook/roberge/Output/GEM5/Olivier/NAM-11m_ERA5_GEM50_PCPTYPEnil/Samples/NAM-11m_ERA5_GEM50_PCPTYPEnil

# path_sim_2p5km_mam = sorted(glob.glob(fr'/pampa/roberge/Output/GEM5/Cascades_CORDEX/CLASS/Safe_versions/Spinup/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon/Samples/ECan_2.5km_NAM11mP3_newP3_CLASS_DEEPoff_SHALon_{end_date.year}0*[3-5]'))
path_sim_2p5km_mam = sorted(glob.glob(fr'/chinook/roberge/Output/GEM5/Olivier/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil/Samples/ECan_2.5km_NAM11mP3_GEM50_PCPTYPEnil_{end_date.year}0*[3-5]'))


season_path = {'ON':{'daymet_tt':path_daymet_tt_on,
                     'daymet_pr':path_daymet_pr_on,
                     '11km':path_sim_11km_on,
                     '2p5km':path_sim_2p5km_on},
               'DJF':{'daymet_tt':path_daymet_tt_djf,
                     'daymet_pr':path_daymet_pr_djf,
                     '11km':path_sim_11km_djf,
                     '2p5km':path_sim_2p5km_djf},
               'MAM':{'daymet_tt':path_daymet_tt_mam,
                     'daymet_pr':path_daymet_pr_mam,
                     '11km':path_sim_11km_mam,
                     '2p5km':path_sim_2p5km_mam}}

# image output dir
image_output_dir = "/upslope/chalifour/projet_maitrise/fig/map_season"

In [5]:
# Domain
xll, yll = m.transform_point(lonE2p5[0, 0], latE2p5[0, 0], ccrs.PlateCarree())
xur, yur = m.transform_point(lonE2p5[-1, -1], latE2p5[-1, -1], ccrs.PlateCarree())

def get_verts_new(categ_box):
    verts = [(categ_box['west'], categ_box['south']),
             (categ_box['west'], categ_box['north']),
             (categ_box['east'], categ_box['north']),
             (categ_box['east'], categ_box['south']),
             (0., 0.)]
    return verts

box = {'west': xll+17, 'east': xur-9, 'north':yur+4 , 'south': yll-1}

vert = get_verts_new(box)
codes = [Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY, ]

path_small = Path(vert, codes)
patch_small = patches.PathPatch(path_small, facecolor='none', ec="black", lw=2, ls='--',
                                        zorder=99999999, label='Studied domain', transform=ccrs.PlateCarree())


In [6]:
path_sim_11km = path_sim_11km_on+path_sim_11km_djf+path_sim_11km_mam

path_sim_2p5km = path_sim_2p5km_on+path_sim_2p5km_djf+path_sim_2p5km_mam

path_saving='/upslope/chalifour/projet_maitrise/data_simulation_monthly'

In [7]:

var_list_simulation = ['PR','TT','UU','VV']
# var_list_simulation = []
get_data = False
# get_data = True

if get_data:
    for idx in range(len(timerange_month.month)-1):

        list_month_tt_2p5km=[]
        list_month_pr_2p5km=[]
        list_month_tt_11km=[]
        list_month_pr_11km=[]
        list_month_uu_2p5km=[]
        list_month_uu_11km=[]
        list_month_vv_2p5km=[]
        list_month_vv_11km=[]
        print(f'Getting {timerange_month[idx].year}-{timerange_month[idx].month} ')

        timerange_hour = pd.date_range(timerange_month[idx].strftime('%Y-%m'),timerange_month[idx+1].strftime('%Y-%m'),freq='1h')

        timerange_hour = timerange_hour[timerange_hour.month == timerange_month[idx].month]

        timerange_hour_datev = []

        for date_h in timerange_hour:
            timerange_hour_datev.append(rpn_chris.date_to_datev(date_h))


        # path_11km = glob.glob(path_sim_11km[idx]+'/*m*')

        # fid_11km = rmn.fstopenall(path_11km,rmn.FST_RO)
        #
        # for j,phase in enumerate(var_list_simulation):
        #     print(f'getting {phase} 11km')
        #     for hidx,datev in enumerate(timerange_hour_datev):
        #         if hidx == 0:
        #             rec_11km = rmn.fstlir(fid_11km, nomvar=phase, datev=timerange_hour_datev[1])
        #             mygrid_11km = rmn.readGrid(fid_11km,rec_11km)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
        #             latlondict_11km = rmn.gdll(mygrid_11km)               # Create 2-D lat and lon fields from the grid information
        #             lat_var_11km = latlondict_11km['lat']                     # Assign 'lat' to 2-D latitude field
        #             lon_var_11km = latlondict_11km['lon']
        #             # np.save(path_saving+f"/11km/lon.npy",np.array(lon_var_11km))
        #
        #             # np.save(path_saving+f"/11km/lat.npy",np.array(lat_var_11km))
        #             if phase == 'TT':
        #                 list_month_tt_11km.append(np.full(np.shape(lon_var_11km),np.nan))
        #             elif phase == 'UU':
        #                 list_month_uu_11km.append(np.full(np.shape(lon_var_11km),np.nan))
        #             elif phase == 'VV':
        #                 list_month_vv_11km.append(np.full(np.shape(lon_var_11km),np.nan))
        #             else:
        #                 list_month_pr_11km.append(np.full(np.shape(lon_var_11km),np.nan))
        #
        #
        #         else:
        #             # key1 = rmn.fstinf(fid, nomvar=phase)
        #             rec_11km = rmn.fstlir(fid_11km, nomvar=phase, datev=datev)
        #
        #             mygrid_11km = rmn.readGrid(fid_11km,rec_11km)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
        #             latlondict_11km = rmn.gdll(mygrid_11km)               # Create 2-D lat and lon fields from the grid information
        #             lat_var_11km = latlondict_11km['lat']                     # Assign 'lat' to 2-D latitude field
        #             lon_var_11km = latlondict_11km['lon']
        #             array_11km = rec_11km['d']
        #
        #             if phase == 'TT':
        #                 list_month_tt_11km.append(array_11km)
        #
        #             elif phase == 'UU':
        #                 list_month_uu_11km.append(array_11km*0.514444)
        #             elif phase == 'VV':
        #                 list_month_vv_11km.append(array_11km*0.514444)
        #             else:
        #                 array_11km = array_11km * 1000 * 3600
        #
        #                 array_11km[array_11km <= 0.1] = 0
        #                 list_month_pr_11km.append(array_11km)
        #
        #
        # # np.save(path_saving+f"/11km/TT_{timerange_month[idx].year}_{timerange_month[idx].month}.npy",np.array(list_month_tt_11km))
        # #
        # # np.save(path_saving+f"/11km/PR_{timerange_month[idx].year}_{timerange_month[idx].month}.npy",np.array(list_month_pr_11km))
        # print()
        # tt_11km = xr.DataArray(data=list_month_tt_11km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_11km),lat=(["x", "y"], lat_var_11km),time=timerange_hour),)
        # pr_11km = xr.DataArray(data=list_month_pr_11km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_11km),lat=(["x", "y"], lat_var_11km),time=timerange_hour))
        # uu_11km = xr.DataArray(data=list_month_uu_11km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_11km),lat=(["x", "y"], lat_var_11km),time=timerange_hour))
        # vv_11km = xr.DataArray(data=list_month_vv_11km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_11km),lat=(["x", "y"], lat_var_11km),time=timerange_hour))
        #
        # with open(path_saving+f"/11km_P3/TT_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
        #           'wb') as f:
        #     pickle.dump(tt_11km, f)
        # tt_11km=[]
        #
        # with open(path_saving+f"/11km_P3/UU_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
        #           'wb') as f:
        #     pickle.dump(uu_11km, f)
        # uu_11km=[]
        #
        # with open(path_saving+f"/11km_P3/VV_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
        #           'wb') as f:
        #     pickle.dump(vv_11km, f)
        # vv_11km=[]
        # with open(path_saving+f"/11km_P3/PR_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
        #           'wb') as f:
        #     pickle.dump(pr_11km, f)
        # #
        # pr_11km=[]
        #
        # rmn.fstcloseall(fid_11km)



        fid_2p5km = rmn.fstopenall(path_sim_2p5km[idx]+'/*m*',rmn.FST_RO)
        for j,phase in enumerate(var_list_simulation):
            print(f'getting {phase} 2.5km')
            for hidx,datev in enumerate(timerange_hour_datev):
                if hidx == 0:
                    rec_2p5km = rmn.fstlir(fid_2p5km, nomvar=phase, datev=timerange_hour_datev[1])
                    mygrid_2p5km = rmn.readGrid(fid_2p5km,rec_2p5km)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
                    latlondict_2p5km = rmn.gdll(mygrid_2p5km)               # Create 2-D lat and lon fields from the grid information
                    lat_var_2p5km = latlondict_2p5km['lat']                     # Assign 'lat' to 2-D latitude field
                    lon_var_2p5km = latlondict_2p5km['lon']
                    # np.save(path_saving+f"/2p5km/lon.npy",np.array(lon_var_2p5km))

                    # np.save(path_saving+f"/2p5km/lat.npy",np.array(lat_var_2p5km))

                    if phase == 'TT':
                        list_month_tt_2p5km.append(np.full(np.shape(lon_var_2p5km),np.nan))
                    elif phase == 'UU':
                        list_month_uu_2p5km.append(np.full(np.shape(lon_var_2p5km),np.nan))
                    elif phase == 'VV':
                        list_month_vv_2p5km.append(np.full(np.shape(lon_var_2p5km),np.nan))

                    else:
                        list_month_pr_2p5km.append(np.full(np.shape(lon_var_2p5km),np.nan))


                else:
                    rec_2p5km = rmn.fstlir(fid_2p5km, nomvar=phase, datev=datev)
                    mygrid_2p5km = rmn.readGrid(fid_2p5km,rec_2p5km)              # Get the grid information for the (LAM) Grid -- Reads the tictac's
                    latlondict_2p5km = rmn.gdll(mygrid_2p5km)               # Create 2-D lat and lon fields from the grid information
                    lat_var_2p5km = latlondict_2p5km['lat']                     # Assign 'lat' to 2-D latitude field
                    lon_var_2p5km = latlondict_2p5km['lon']
                    array_2p5km = rec_2p5km['d']

                    if phase == 'TT':
                        list_month_tt_2p5km.append(array_2p5km)
                    elif phase == 'UU':
                        list_month_uu_2p5km.append(array_2p5km*0.514444)
                    elif phase == 'VV':
                        list_month_vv_2p5km.append(array_2p5km*0.514444)
                    else:
                        array_2p5km = array_2p5km* 1000* 3600
                        array_2p5km[array_2p5km <= 0.1] = 0
                        list_month_pr_2p5km.append(array_2p5km)



        tt_2p5km = xr.DataArray(data=list_month_tt_2p5km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_2p5km),lat=(["x", "y"], lat_var_2p5km),time=timerange_hour))
        uu_2p5km = xr.DataArray(data=list_month_uu_2p5km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_2p5km),lat=(["x", "y"], lat_var_2p5km),time=timerange_hour))

        vv_2p5km = xr.DataArray(data=list_month_vv_2p5km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_2p5km),lat=(["x", "y"], lat_var_2p5km),time=timerange_hour))

        with open(path_saving+f"/2p5km_P3/TT_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
                  'wb') as f:
            pickle.dump(tt_2p5km, f)
        tt_2p5km = []

        with open(path_saving+f"/2p5km_P3/UU_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
                  'wb') as f:
            pickle.dump(uu_2p5km, f)
        uu_2p5km=[]

        with open(path_saving+f"/2p5km_P3/VV_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
                  'wb') as f:
            pickle.dump(vv_2p5km, f)
        vv_2p5km=[]
        # np.save(path_saving+f"/2p5km/TT_{timerange_month[idx].year}_{timerange_month[idx].month}.npy",np.array(list_month_tt_2p5km))
        pr_2p5km = xr.DataArray(data=list_month_pr_2p5km,dims=['time',"x", "y"],coords=dict(lon=(["x", "y"], lon_var_2p5km),lat=(["x", "y"], lat_var_2p5km),time=timerange_hour))
        #
        with open(path_saving+f"/2p5km_P3/PR_{timerange_month[idx].year}_{timerange_month[idx].month}.pickle",
                  'wb') as f:
            pickle.dump(pr_2p5km, f)
        # np.save(path_saving+f"/2p5km/PR_{timerange_month[idx].year}_{timerange_month[idx].month}.npy",np.array(list_month_pr_2p5km))
        pr_2p5km=[]

        rmn.fstcloseall(fid_2p5km)



# Get data to plot
## Daymet

In [8]:
print('Data Daymet')
# daymet_tt = xr.open_mfdataset(path_daymet_tt_on+path_daymet_tt_djf+path_daymet_tt_mam, combine='by_coords', concat_dim="time")
# daymet_pr = xr.open_mfdataset(path_daymet_pr_on+path_daymet_pr_djf+path_daymet_pr_mam, combine='by_coords', concat_dim="time")

# print('Data Daymet DJF')
# daymet_tt_djf = xr.open_mfdataset(path_daymet_tt_djf, combine='by_coords', concat_dim="time")
# daymet_pr_djf = xr.open_mfdataset(path_daymet_pr_djf, combine='by_coords', concat_dim="time")

# print('Data Daymet MAM')
# daymet_tt_mam = xr.open_mfdataset(path_daymet_tt_mam, combine='by_coords', concat_dim="time")
# daymet_pr_mam = xr.open_mfdataset(path_daymet_pr_mam, combine='by_coords', concat_dim="time")



Data Daymet


In [9]:
# cut to domain

get_daymet = True
# get_daymet = False

# day_tt = xr.concat([daymet_tt_on,daymet_tt_djf,daymet_tt_mam],dim='time').tavg
# daymet_tt_on,daymet_tt_djf,daymet_tt_mam = [],[],[]
# day_pr = xr.concat([daymet_pr_on,daymet_pr_djf,daymet_pr_mam],dim='time').prcp
# daymet_pr_on,daymet_pr_djf,daymet_pr_mam = [],[],[]
list_var = ['tavg']
mask_day = 0
if get_daymet:
    list_path_day = [path_daymet_tt_on+path_daymet_tt_djf+path_daymet_tt_mam,path_daymet_pr_on+path_daymet_pr_djf+path_daymet_pr_mam]
    for i,var in enumerate(list_var):
        day_path = list_path_day[i]
        for j,path in enumerate(day_path):
            print(path)
            if i == 0 and j ==0:
                day_var = xr.open_mfdataset(path)[var]
                print(f'Treating {var}')
                grid_lonlat = m.transform_points(ccrs.PlateCarree(),day_var.lon.values,day_var.lat.values)[:,:,0:2]


                xll, yll = m.transform_point(lonE2p5[0, 0], latE2p5[0, 0], ccrs.PlateCarree())
                xur, yur = m.transform_point(lonE2p5[-1, -1], latE2p5[-1, -1], ccrs.PlateCarree())

                box = {'west': xll+ 17, 'east': xur- 9, 'north':yur- 1 , 'south': yll+ 4}

                vert = get_verts_new(box)
                codes = [Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY, ]
                path_small = Path(vert, codes)

                shape_grid = np.shape(grid_lonlat)
                point_in_domain = path_small.contains_points(grid_lonlat.reshape(shape_grid[0] * shape_grid[1], 2),radius=0).reshape(shape_grid[0], shape_grid[1])

                mask_day = point_in_domain
                day_var.load()
                data_day_mask = day_var.where(mask_day).dropna(dim='x',how="all").dropna(dim='y',how="all")

            else:
                day_var = xr.open_mfdataset(path, combine='by_coords', concat_dim="time")[list_var[i]]
                day_var.load()
                data_day_mask = day_var.where(mask_day).dropna(dim='x',how="all").dropna(dim='y',how="all")


            print(data_day_mask.min().values)
            print(data_day_mask.max().values)
            data_day_mask.to_netcdf(path_saving+f"/daymet/{var}_{path[-12:-6]}.nc")

    # daymet_tt_on_mean = daymet_tt_on
    # daymet_tt_djf_mean = daymet_tt_djf
    # daymet_tt_mam_mean = daymet_tt_mam
    #
    #
    # daymet_pr_on_sum = daymet_pr_on
    # daymet_pr_djf_sum = daymet_pr_djf
    # daymet_pr_mam_sum = daymet_pr_mam


    # print('Treating TT')
    #
    # daymet_tt_on_mean = ma.masked_array(daymet_tt_on_mean['tavg'].values, mask)
    #
    # xrdata_tt_on = xr.DataArray(data=daymet_tt_on_mean['tavg'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_tt_on_mean.to_netcdf(path_saving+f"/daymet/TT_on_{begin_date.year}.nc")
    #
    # daymet_tt_djf_mean = ma.masked_array(daymet_tt_djf_mean['tavg'].values, mask)
    # xrdata_tt_djf = xr.DataArray(data=daymet_tt_djf_mean['tavg'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_tt_djf_mean.to_netcdf(path_saving+f"/daymet/TT_djf_{begin_date.year}.nc")
    #
    # daymet_tt_mam_mean = ma.masked_array(daymet_tt_mam_mean['tavg'].values, mask)
    # xrdata_tt_mam = xr.DataArray(data=daymet_tt_mam_mean['tavg'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_tt_mam_mean.to_netcdf(path_saving+f"/daymet/TT_mam_{begin_date.year}.nc")

    # print('Treating PR')
    # daymet_pr_on_sum = ma.masked_array(daymet_pr_on_sum['prcp'].values, mask)
    # xrdata_pr_on = xr.DataArray(data=daymet_pr_on_sum['prcp'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_pr_on_sum.to_netcdf(path_saving+f"/daymet/PR_on_{begin_date.year}.nc")

    # daymet_pr_djf_sum = ma.masked_array(daymet_pr_djf_sum['prcp'].values, mask)
    # xrdata_pr_djf = xr.DataArray(data=daymet_pr_djf_sum['prcp'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_pr_djf_sum.to_netcdf(path_saving+f"/daymet/PR_djf_{begin_date.year}.nc")
    #
    # daymet_pr_mam_sum = ma.masked_array(daymet_pr_mam_sum['prcp'].values, mask)
    # xrdata_pr_mam = xr.DataArray(data=daymet_pr_mam_sum['prcp'].values,
    #                                  dims=["x", "y", ],
    #                                  coords=dict(lon=(["x", "y"], daymet_tt_on.lon.values),
    #                                              lat=(["x", "y"], daymet_tt_on.lat.values),
    #                                              ))
    # daymet_pr_mam_sum.to_netcdf(path_saving+f"/daymet/PR_mam_{begin_date.year}.nc")

else:
    xrdata_tt_on = xr.open_mfdataset(path_saving+f"/daymet/TT_on_{begin_date.year}.nc")
    xrdata_tt_djf = xr.open_mfdataset(path_saving+f"/daymet/TT_djf_{begin_date.year}.nc")
    xrdata_tt_mam = xr.open_mfdataset(path_saving+f"/daymet/TT_mam_{begin_date.year}.nc")

    xrdata_pr_on = xr.open_mfdataset(path_saving+f"/daymet/PR_on_{begin_date.year}.nc")
    xrdata_pr_djf = xr.open_mfdataset(path_saving+f"/daymet/PR_djf_{begin_date.year}.nc")
    xrdata_pr_mam = xr.open_mfdataset(path_saving+f"/daymet/PR_mam_{begin_date.year}.nc")


/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2021/10/daymet_1km_v4r1_tavg_lc_202110_d.nc4
Treating tavg
-6.430000305175781
19.415000915527344
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2021/11/daymet_1km_v4r1_tavg_lc_202111_d.nc4
-16.744998931884766
12.135000228881836
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2021/12/daymet_1km_v4r1_tavg_lc_202112_d.nc4
-32.96500015258789
10.809999465942383
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2022/01/daymet_1km_v4r1_tavg_lc_202201_d.nc4
-35.220001220703125
5.180000305175781
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2022/02/daymet_1km_v4r1_tavg_lc_202202_d.nc4
-38.48500061035156
10.11500072479248
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2022/03/daymet_1km_v4r1_tavg_lc_202203_d.nc4
-35.70000076293945
11.984999656677246
/home/archive/OBSERVATIONS/Daymet_1km_v4r1/daily/tavg/lc/nc4/2022/04/daymet_1km_v4r1_tavg_lc_202204_d.nc4
-16.899999618530

 ## 11km

In [10]:
list_tt_11km=[]
list_pr_11km=[]
for month in timerange_month[:-1]:
    print(f'Loading {month.year}-{month.month}')
    with open(f'/upslope/chalifour/projet_maitrise/data_simulation_monthly/11km_P3/TT_{month.year}_{month.month}.pickle', 'rb') as pickle_file_tt:
        tt_11km_month = pickle.load(pickle_file_tt)
        max_tt_11km_daily= tt_11km_month.resample(time="1D").max()
        min_tt_11km_daily= tt_11km_month.resample(time="1D").min()
        mean_tt_11km_daily = (max_tt_11km_daily+min_tt_11km_daily)/2
        list_tt_11km.append(mean_tt_11km_daily)

    with open(f'/upslope/chalifour/projet_maitrise/data_simulation_monthly/11km_P3/PR_{month.year}_{month.month}.pickle', 'rb') as pickle_file_pr:
        pr_11km_month = pickle.load(pickle_file_pr)
        sum_pr_11km_daily = pr_11km_month.resample(time="1D").sum()
        list_pr_11km.append(sum_pr_11km_daily)


tt_11km = xr.concat(list_tt_11km,dim='time')
pr_11km = xr.concat(list_pr_11km,dim='time')

Loading 2021-10
Loading 2021-11


KeyboardInterrupt: 

In [ ]:

dict_season_tt_11km={}
dict_season_pr_11km={}
for month_id, ds_month in tt_11km.groupby('time.season'):
    ds_month_complete = ds_month.mean(dim='time',skipna=True)

    # xl = m.transform_points(ccrs.PlateCarree(),ds_month_complete.lon.values-360, ds_month_complete.lat.values)
    #
    # lon_m_ds_month_compl = xl[:,:,0]
    # lat_m_ds_month_compl = xl[:,:,1]
    #
    # grid = np.stack((lon_m_ds_month_compl, lat_m_ds_month_compl), axis=-1)
    # shape_grid = np.shape(grid)
    #
    # point_in_domain = path_small.contains_points(grid.reshape(shape_grid[0] * shape_grid[1], 2),
    #                                              radius=1e-9).reshape(shape_grid[0], shape_grid[1])
    #
    # mask = np.invert(point_in_domain)
    # lon_daymet_mask = ma.masked_array(ds_month_complete.lon.values, mask)
    # lat_daymet_mask = ma.masked_array(ds_month_complete.lat.values, mask)

    # ds_month_season = xr.DataArray(data=ds_month_complete.values,
    #                              dims=["x", "y", ],
    #                              coords=dict(lon=(["x", "y"], ds_month_complete.lon.values),
    #                                          lat=(["x", "y"], ds_month_complete.lat.values),
    #                                          ))
    dict_season_tt_11km[month_id] = ds_month_complete

with open(path_saving+f"/11km_P3/TT_season_dict_{begin_date.year}.pickle",'wb') as f:
    pickle.dump(dict_season_tt_11km, f)

for month_id, ds_month in pr_11km.groupby('time.season'):
    ds_month_complete = ds_month.sum(dim='time',skipna=True)
    # xl = m.transform_points(ccrs.PlateCarree(),ds_month_complete.lon.values-360, ds_month_complete.lat.values)
    #
    # lon_m_ds_month_compl = xl[:,:,0]
    # lat_m_ds_month_compl = xl[:,:,1]
    #
    # grid = np.stack((lon_m_ds_month_compl, lat_m_ds_month_compl), axis=-1)
    # shape_grid = np.shape(grid)
    #
    # point_in_domain = path_small.contains_points(grid.reshape(shape_grid[0] * shape_grid[1], 2),
    #                                              radius=1e-9).reshape(shape_grid[0], shape_grid[1])
    #
    # mask = np.invert(point_in_domain)
    # lon_daymet_mask = ma.masked_array(ds_month_complete.lon.values, mask)
    # lat_daymet_mask = ma.masked_array(ds_month_complete.lat.values, mask)

    # ds_month_season = xr.DataArray(data=ds_month_complete.values,
    #                              dims=["x", "y", ],
    #                              coords=dict(lon=(["x", "y"], ds_month_complete.lon.values),
    #                                          lat=(["x", "y"], ds_month_complete.lat.values),
    #                                          ))
    dict_season_pr_11km[month_id] = ds_month_complete
with open(path_saving+f"/11km_P3/PR_season_dict_{begin_date.year}.pickle",'wb') as f:
    pickle.dump(dict_season_pr_11km, f)


## 2.5 km

In [ ]:
list_tt_2p5km = []
list_pr_2p5km = []
for month in timerange_month[:-1]:
    print(f'Loading {month.year}-{month.month}')
    with open(f'/upslope/chalifour/projet_maitrise/data_simulation_monthly/2p5km_P3/TT_{month.year}_{month.month}.pickle',
              'rb') as pickle_file_tt:
        tt_2p5km_month = pickle.load(pickle_file_tt)
        max_tt_2p5km_daily = tt_2p5km_month.resample(time="1D").max()
        min_tt_2p5km_daily = tt_2p5km_month.resample(time="1D").min()
        mean_tt_2p5km_daily = (max_tt_2p5km_daily + min_tt_2p5km_daily) / 2
        list_tt_2p5km.append(mean_tt_2p5km_daily)

    with open(f'/upslope/chalifour/projet_maitrise/data_simulation_monthly/2p5km_P3/PR_{month.year}_{month.month}.pickle',
              'rb') as pickle_file_pr:
        tt_2p5km_month = pickle.load(pickle_file_pr)
        sum_pr_2p5km_daily = tt_2p5km_month.resample(time="1D").sum()
        list_pr_2p5km.append(sum_pr_2p5km_daily)

tt_2p5km = xr.concat(list_tt_2p5km, dim='time')

pr_2p5km = xr.concat(list_pr_2p5km, dim='time')

In [ ]:

dict_season_tt_2p5km = {}
dict_season_pr_2p5km = {}

for month_id, ds_month in tt_2p5km.groupby('time.season'):
    ds_month_complete = ds_month.mean(dim='time',skipna=True)

    # xl = m.transform_points(ccrs.PlateCarree(),ds_month_complete.lon.values-360, ds_month_complete.lat.values)
    #
    # lon_m_ds_month_compl = xl[:,:,0]
    # lat_m_ds_month_compl = xl[:,:,1]
    #
    # grid = np.stack((lon_m_ds_month_compl, lat_m_ds_month_compl), axis=-1)
    # shape_grid = np.shape(grid)
    #
    # point_in_domain = path_small.contains_points(grid.reshape(shape_grid[0] * shape_grid[1], 2),
    #                                              radius=1e-9).reshape(shape_grid[0], shape_grid[1])
    #
    # mask = np.invert(point_in_domain)
    # lon_daymet_mask = ma.masked_array(ds_month_complete.lon.values, mask)
    # lat_daymet_mask = ma.masked_array(ds_month_complete.lat.values, mask)
    #
    # ds_month_season = xr.DataArray(data=ma.masked_array(ds_month_complete.values, mask),
    #                                dims=["x", "y", ],
    #                                coords=dict(lon=(["x", "y"], lon_daymet_mask),
    #                                            lat=(["x", "y"], lat_daymet_mask),
    #                                            ))
    dict_season_tt_2p5km[month_id] = ds_month_complete
with open(path_saving+f"/2p5km_P3/TT_season_dict_{begin_date.year}.pickle",'wb') as f:
    pickle.dump(dict_season_tt_2p5km, f)
for month_id, ds_month in pr_2p5km.groupby('time.season'):
    ds_month_complete = ds_month.sum(dim='time',skipna=True)

    # xl = m.transform_points(ccrs.PlateCarree(),ds_month_complete.lon.values-360, ds_month_complete.lat.values)
    #
    # lon_m_ds_month_compl = xl[:,:,0]
    # lat_m_ds_month_compl = xl[:,:,1]
    #
    # grid = np.stack((lon_m_ds_month_compl, lat_m_ds_month_compl), axis=-1)
    # shape_grid = np.shape(grid)
    #
    # point_in_domain = path_small.contains_points(grid.reshape(shape_grid[0] * shape_grid[1], 2),
    #                                              radius=1e-9).reshape(shape_grid[0], shape_grid[1])
    #
    # mask = np.invert(point_in_domain)
    # lon_daymet_mask = ma.masked_array(ds_month_complete.lon.values, mask)
    # lat_daymet_mask = ma.masked_array(ds_month_complete.lat.values, mask)
    #
    # ds_month_season = xr.DataArray(data=ma.masked_array(ds_month_complete.values, mask),
    #                                dims=["x", "y", ],
    #                                coords=dict(lon=(["x", "y"], lon_daymet_mask),
    #                                            lat=(["x", "y"], lat_daymet_mask),
    #                                            ))
    dict_season_pr_2p5km[month_id] = ds_month_complete
with open(path_saving+f"/2p5km_P3/PR_season_dict_{begin_date.year}.pickle",'wb') as f:
    pickle.dump(dict_season_pr_2p5km, f)